# 準備數據

In [1]:
import cv2
import numpy as np 

In [2]:
image = cv2.imread('images/digits.png')
gray = cv2.cvtColor(image,cv2.COLOR_BGR2GRAY)
small = cv2.pyrDown(image) #. 縮小了四分之一
image.shape

(1000, 2000, 3)

In [3]:
# cv2.imshow("digits image",image)
# cv2.waitKey(0)

In [4]:
# 把图像分割成 5000 每个的大小事 20 * 20(每个数字的大小)
# 这给了我们一个四维的矩阵 [50,100, 20,20]
cells = [np.hsplit(row,100) for row in np.vsplit(gray,50)]

# for row in np.vsplit(gray,50):
#     np.hsplit(row,100)

In [5]:
x = np.array(cells)
print ("The shape of our cells array: " + str(x.shape))

The shape of our cells array: (50, 100, 20, 20)


# 數據分割 

In [6]:
train = x [:,:70].reshape(-1, 400).astype(np.float32) # (3500,400)
test = x [:,70:100].reshape(-1, 400).astype(np.float32) # (1500,400)

In [7]:
test.shape, train.shape

((1500, 400), (3500, 400))

# 创建一个标签 告诉他这是什么鬼？

In [8]:
k = [0,1,2,3,4,5,6,7,8,9]
train_labels = np.repeat(k, 350)[:,np.newaxis]
test_labels = np.repeat(k, 150)[:,np.newaxis]

In [9]:
# KNN 算法 

In [10]:
knn = cv2.KNearest()
knn.train(train,train_labels) # 把训练数据 标上 0 ~ 9 的标记
ret, result, neghbors, distance = knn.find_nearest(test, k =3) # k值 等於三 来计算前面的模型

In [11]:
# 检查分类的准确性

In [12]:
matches = result == test_labels # 验证的结果 相当于 result 也等于 matches
correct = np.count_nonzero(matches)
accuracy = correct * (100.0 / result.size)

In [13]:
print ("Accuracy is %.2f" % accuracy + "%")

Accuracy is 93.47%


# 准备一些 我们等一下要用的函数

In [14]:
import cv2
import numpy as np 

def x_cord_contour(contour): # X 數據的輪廓 取出我们需要的数字
    
    if cv2.contourArea(contour) > 10:
        M = cv2.moments(contour)
        return(int(M['m10'] / M['m00']))
    
def makeSquare(not_square): # 取出所選擇出來的圖片
    BLACK  = [0,0,0] # 黑色的部分都是 0
    img_dim = not_square.shape #  把除了 0 以外的部分序列化 会成为一个 二维数组 [高,宽]
    height = img_dim[0]
    width = img_dim[1]
    if (height == width):
        square = not_square
        return square
    else: 
        # 把圖片放大兩倍 方便識別
        doublesize = cv2.resize(not_square,(2*width, 2* height), interpolation = cv2.INTER_CUBIC)# interpolation 是一种图像大小的转换，把原本的图片做参数里面的改变 INTER_CUBIC 兩倍大小
        height = 2 * height
        width = 2 * width
        # 把空白的地方 用黑色填滿 BLACK
        if (height > width):
            pad = (height - width)/2
            #print("Padding = ", pad)
            doublesize_square = cv2.copyMakeBorder(doublesize,0,0,pad,\
                                                   pad,cv2.BORDER_CONSTANT,value=BLACK)
        else:
            pad = (width - height)/2
            #print("Padding = ", pad)
            doublesize_square = cv2.copyMakeBorder(doublesize,pad,pad,0,0,\
                                                   cv2.BORDER_CONSTANT,value=BLACK)
        doublesize_square_dim = doublesize_square.shape
        #print("Sq Height = ", doublesize_square_dim[0], "Sq Width = ", doublesize_square_dim[1])
        return doublesize_square
    
def resize_to_pixel(dimensions, image): #  把圖片調整到像素
    # This function then re-sizes an image to the specificied dimenions
    
    buffer_pix = 4 # 設置緩衝
    dimensions  = dimensions - buffer_pix
    squared = image
    r = float(dimensions) / squared.shape[1]
    dim = (dimensions, int(squared.shape[0] * r))
    resized = cv2.resize(image, dim, interpolation = cv2.INTER_AREA)
    img_dim2 = resized.shape
    height_r = img_dim2[0]
    width_r = img_dim2[1]
    BLACK = [0,0,0]
    if (height_r > width_r):
        resized = cv2.copyMakeBorder(resized,0,0,0,1,cv2.BORDER_CONSTANT,value=BLACK)
    if (height_r < width_r):
        resized = cv2.copyMakeBorder(resized,1,0,0,0,cv2.BORDER_CONSTANT,value=BLACK)
    p = 2
    ReSizedImg = cv2.copyMakeBorder(resized,p,p,p,p,cv2.BORDER_CONSTANT,value=BLACK)
    img_dim = ReSizedImg.shape
    height = img_dim[0]
    width = img_dim[1]
    #print("Padded Height = ", height, "Width = ", width)
    return ReSizedImg
    

# 加入手写数字的图片 来看看效果

In [ ]:
import numpy as np
import cv2

image = cv2.imread('images/numbers.jpg')
gray = cv2.cvtColor(image,cv2.COLOR_BGR2GRAY)
cv2.imshow("image", image)
cv2.imshow("gray", gray)
cv2.waitKey(0)

blurred = cv2.GaussianBlur(gray, (5, 5), 0)
cv2.imshow("blurred", blurred)
cv2.waitKey(0)

edged = cv2.Canny(blurred, 30, 150)
cv2.imshow("edged", edged)
cv2.waitKey(0)

# Fint Contours
contours, _ = cv2.findContours(edged.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

#Sort out contours left to right by using their x cordinates
contours = sorted(contours, key = x_cord_contour, reverse = False)

# Create empty array to store entire number
full_number = []

# loop over the contours
for c in contours:
    # compute the bounding box for the rectangle
    (x, y, w, h) = cv2.boundingRect(c)    
    
    #cv2.drawContours(image, contours, -1, (0,255,0), 3)
    #cv2.imshow("Contours", image)

    if w >= 5 and h >= 25:
        roi = blurred[y:y + h, x:x + w]
        ret, roi = cv2.threshold(roi, 127, 255,cv2.THRESH_BINARY_INV)
        squared = makeSquare(roi)
        final = resize_to_pixel(20, squared)
        cv2.imshow("final", final)
        final_array = final.reshape((1,400))
        final_array = final_array.astype(np.float32)
        ret, result, neighbours, dist = knn.find_nearest(final_array, k=1)
        number = str(int(float(result[0])))
        full_number.append(number)
        # draw a rectangle around the digit, the show what the
        # digit was classified as
        cv2.rectangle(image, (x, y), (x + w, y + h), (0, 0, 255), 2)
        cv2.putText(image, number, (x , y + 155),
            cv2.FONT_HERSHEY_COMPLEX, 2, (255, 0, 0), 2)
        cv2.imshow("image", image)
        cv2.waitKey(0) 
        
cv2.destroyAllWindows()
print ("The number is: " + ''.join(full_number))

